# Documentation: Topology

The topology of all elements implemented on NeoPZ are grouped in pztopology. The classes associated to this namespace are listed in the following:

 
1. TPZPoint
2. TPZLine
3. TPZQuadrilateral
4. TPZTriangle
5. TPZCube
6. TPZPrism
7. TPZPyramid
8. TPZTetrahedron

Each element (e.g. point, line, quadrilateral, triangle, etc) is considered as the union of its sides. Each side is composed of an open set of points ans has inherent dimension and topology.

There is a unique side whose closure is the proper element. By convention this is the highest numbered side.

To each element a parameter space can be associated which is commonly known as the master element space.

## Main atributions of the class Topology

### a. Definition of number of corners, sides, dimension and faces.

Side are open sets of points. Within NeoPZ sides are associated with typical finite element topologies: point, line, quadrilateral, triangle, tetrahedra, hexahedra, prism, pyramid.

An element is a closed set of points. Each element can be partioned into its sides (an intersection os its sides is empty and their union is the element).

Zero dimensional sides are called corners. A corner is both open and closed set.

In [3]:
// Make sure to change '/usr/local/' according to the installation in your computer
#pragma cling add_include_path("/usr/local/pzlib/include")
#pragma cling add_include_path("/usr/local/pzlib/include/Topology")
#pragma cling add_include_path("/usr/local/pzlib/include/Matrix")
#pragma cling add_include_path("/usr/local/pzlib/include/Util")
#pragma cling add_include_path("/usr/local/pzlib/include/Common")
#pragma cling add_include_path("/usr/local/pzlib/include/Save")
#pragma cling add_include_path("/usr/local/pzlib/include/Integral")
#pragma cling add_include_path("/usr/local/pzlib/include/LinearSolver")
#pragma cling add_include_path("/usr/local/pzlib/include/PerfUtil")

#pragma cling add_library_path("/usr/local/lib/")

//#pragma cling load("libpz.so")


#include <iostream>

#include "pzvec.h"
//#include "tpzintpoints.h"
//#include "pzquad.h"
#include "tpzquadrilateral.h"

TPZVec<REAL> blob;
blob.resize(3);
blob[0] = 10.;
std::cout << blob[0] << std::endl;

// Link error to be solved:
pztopology::TPZQuadrilateral quad;
std::cout << quad.NCornerNodes << std::endl;
std::cout << quad.NSides << std::endl;
std::cout << quad.Dimension << std::endl;
std::cout << quad.NFaces << std::endl;

IncrementalExecutor::executeFunction: symbol '_ZTVN10pztopology16TPZQuadrilateralE' unresolved while linking function '_GLOBAL__sub_I_cling_module_7'!
You are probably missing the definition of vtable for pztopology::TPZQuadrilateral
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol '_ZTTN10pztopology16TPZQuadrilateralE' unresolved while linking function '_GLOBAL__sub_I_cling_module_7'!
You are probably missing the definition of VTT for pztopology::TPZQuadrilateral
Maybe you need to load the corresponding shared library?


4
9
2
4


### b. Definition of the dimension of each side and associated corner nodes.

|    Topology   | Dimension | NCornerNodes |
|:-------------:|:---------:|:------------:|
|     Point     |     0     |       1      |
|      Line     |     1     |       2      |
|    Triangle   |     2     |       3      |
| Quadrilateral |     2     |       4      |
|   Tetrahedra  |     3     |       4      |
|   Hexahedra   |     3     |       8      |
|     Prism     |     3     |       6      |
|    Pyramid    |     3     |       5      |

In [2]:
int side = 2;
int dimside = quad.SideDimension(side);
std::cout << "The dimension of the side " << side << " is" << dimside << std::endl;
int nsidenodes = quad.NSideNodes(side);
std::cout << "The number of associated corner nodes of the side " << side << " is " << nsidenodes << std::endl;

IncrementalExecutor::executeFunction: symbol '_ZN10pztopology16TPZQuadrilateral13SideDimensionEi' unresolved while linking function '_GLOBAL__sub_I_cling_module_8'!
You are probably missing the definition of pztopology::TPZQuadrilateral::SideDimension(int)
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol '_ZN10pztopology16TPZQuadrilateral10NSideNodesEi' unresolved while linking function '_GLOBAL__sub_I_cling_module_8'!
You are probably missing the definition of pztopology::TPZQuadrilateral::NSideNodes(int)
Maybe you need to load the corresponding shared library?


The number of associated corner nodes of the side 2 is 0


### c. Definition of the parametric transformation between sides (note that this function returns a TPZTransform).

Each element is partitioned by its sides. The closure of a side includes a number os sides and forms an element topology. This imples that each point on the boundary of a side belongs to a side of lower dimension.

A point in the parametric domain of a side can be transformed to a point in the parametric domain of the element. The transformation is a homogeneous transformation (i.e. a linear transformation followed by a translation).

These functionalities are implemented by the method SideToSideTransform. It is assumed that sidefrom belongs to sideto.

If sidefrom corresponds to the volume of the element, the method will return a transformation that is a projection

In [3]:
# int sidefrom = 2;
# int sideto = 1;
# TPZTransform<> tr = quad.SideToSideTransform(sidefrom, sideto);

### d. Creation of integration rules associated to each side (note that the integration rule is a pointer to the TPZIntPoints type).

In [4]:
# int integr_order = 3;
# TPZIntPoints * integr = quad.CreateSideIntegrationRule(side, integr_order)

### e. Definition of a transformation index associated with a side.

In [5]:
# TPZVec<int64_t> id;
# int transf_id = quad.GetTransformId(id);

### f. Definition of the permutation index associated with the element.

In [6]:
# TPZVec<int> permgather; // output variable
# quad.GetSideHDivPermutation(transf_id, permgather);

### g. Relationship between sides: which sides are included in the closure of a side and orientation (note that the output variable is a stack object).

In [7]:
# TPZStack<int> high;
# quad.HigherDimensionSides(side, high);

### h. Projection of a point to a side (this function is actually implemented on TPZShape).

In [8]:
# int rib;
# TPZVec<REAL> in;
# REAL out;
# pzshape::TPZShapeQuad::ProjectPoint2dQuadToRib(rib, in, out);